In [1]:
import pandas as pd
import numpy as np

import re
import nltk

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

import annoy
from gensim.models import Word2Vec, FastText

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

База данных состоит из 12 столбцов:

- id: уникальный номер сообщения в системе twitter;
- tdate: дата публикации сообщения (твита);
- tmane: имя пользователя, опубликовавшего сообщение;
- ttext:  текст сообщения (твита);
- ttype: поле в котором в дальнейшем будет указано к кому классу относится твит (положительный, отрицательный, нейтральный);
- trep: количество реплаев к данному сообщению. В настоящий момент API твиттера не отдает эту информацию;
- trtw: количество ретвитов к данному сообщению. 
- tfav: число сколько раз данное сообщение было добавлено в избранное другими пользователями;
- tstcount: число всех сообщений пользователя в сети twitter;
- tfoll: количество фоловеров пользователя (тех людей, которые читают пользователя);
- tfrien: количество друзей пользователя (те люди, которых читает пользователь);
- listcount: количество листов-подписок в которые добавлен твиттер-пользователь.

In [2]:
col_names = ['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep', 'trtw', 'tfav', 'tstcount', 'tfoll', 'tfrien', 'listcount']

In [3]:
positive_df = pd.read_csv('positive.csv', sep=';', header=None, index_col=0, names=col_names, encoding='utf-8')
positive_df.head()

,tdate,tmane,ttext,ttype,trep,trtw,tfav,tstcount,tfoll,tfrien,listcount
id,,,,,,,,,,,
408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [4]:
positive_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114911 entries, 408906692374446080 to 411368857035898880
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   tdate      114911 non-null  int64 
 1   tmane      114911 non-null  object
 2   ttext      114911 non-null  object
 3   ttype      114911 non-null  int64 
 4   trep       114911 non-null  int64 
 5   trtw       114911 non-null  int64 
 6   tfav       114911 non-null  int64 
 7   tstcount   114911 non-null  int64 
 8   tfoll      114911 non-null  int64 
 9   tfrien     114911 non-null  int64 
 10  listcount  114911 non-null  int64 
dtypes: int64(9), object(2)
memory usage: 10.5+ MB


In [5]:
len(positive_df)

114911

In [6]:
positive_df.value_counts('ttype')

ttype
1    114911
dtype: int64

In [7]:
negative_df = pd.read_csv('negative.csv', sep=';', header=None, index_col=0, names=col_names, encoding='utf-8')
negative_df.head()

,tdate,tmane,ttext,ttype,trep,trtw,tfav,tstcount,tfoll,tfrien,listcount
id,,,,,,,,,,,
408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [8]:
negative_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111923 entries, 408906762813579264 to 425138595251625984
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   tdate      111923 non-null  int64 
 1   tmane      111923 non-null  object
 2   ttext      111923 non-null  object
 3   ttype      111923 non-null  int64 
 4   trep       111923 non-null  int64 
 5   trtw       111923 non-null  int64 
 6   tfav       111923 non-null  int64 
 7   tstcount   111923 non-null  int64 
 8   tfoll      111923 non-null  int64 
 9   tfrien     111923 non-null  int64 
 10  listcount  111923 non-null  int64 
dtypes: int64(9), object(2)
memory usage: 10.2+ MB


In [9]:
len(negative_df)

111923

In [10]:
negative_df.value_counts('ttype')

ttype
-1    111923
dtype: int64

In [11]:
combine_df = positive_df.iloc[:, 2].append(negative_df.iloc[:, 2], ignore_index = True)
combine_df = combine_df.dropna().drop_duplicates()
combine_df.head()

0    @first_timee хоть я и школота, но поверь, у на...
1    Да, все-таки он немного похож на него. Но мой ...
2    RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3    RT @digger2912: "Кто то в углу сидит и погибае...
4    @irina_dyshkant Вот что значит страшилка :D\nН...
Name: ttext, dtype: object

### 1. Предобработка

In [12]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
morpher = MorphAnalyzer()
sw_ru = set(get_stop_words("ru"))

In [13]:
#1. избавимся от букв латинского алфавита, чисел, знаков препинания и всех символов, например, символ @ встречается почти везде;
#2. разобьем пост на токены;
#3. проведем лемматизацияю, получив нормальную (начальную) форму слова;
#4. удалим стоп-слова.
def preprocess_txt(doc):
    #1. Удалим @user из всех твитов с помощью паттерна "@[\w]*".
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in sw_ru:
            token = token.strip()
            token = morpher.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

In [14]:
combine_df[:5].apply(preprocess_txt)

0    [школотый, поверь, самый, общество, профилиров...
1              [да, таки, похожий, но, мальчик, равно]
2                            [ну, идиотка, испугаться]
3    [кто, угол, сидеть, погибать, голод, порция, в...
4    [вот, страшилка, но, блин, посмотреть, часть, ...
Name: ttext, dtype: object

In [15]:
%%time
data = combine_df.apply(preprocess_txt)
data = data.dropna()

Wall time: 4min 23s


In [16]:
data

0         [школотый, поверь, самый, общество, профилиров...
1                   [да, таки, похожий, но, мальчик, равно]
2                                 [ну, идиотка, испугаться]
3         [кто, угол, сидеть, погибать, голод, порция, в...
4         [вот, страшилка, но, блин, посмотреть, часть, ...
                                ...                        
226829                             [но, хотеть, исправлять]
226830           [скучать, вправлять, мозг, равно, скучать]
226831                            [вот, школа, говно, идти]
226832                           [тауриэль, грусть, обнять]
226833    [такси, везти, работа, раздумывать, приплатить...
Name: ttext, Length: 198708, dtype: object

In [17]:
data.to_pickle('data_prepared.pkl')

### 2. Word2Vec

In [18]:
w2v_model = Word2Vec(
    min_count=10,
    window=2,
    vector_size=300,
    negative=10,
    alpha=0.03,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1)

In [19]:
w2v_model.build_vocab(data)

In [20]:
%%time
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=50, report_delay=1)

Wall time: 2min 4s


(34319449, 66831650)

### 3. FastText

In [21]:
ft_model = FastText(
    min_count=10,
    window=2,
    vector_size=300,
    negative=10)

In [22]:
ft_model.build_vocab(data)

In [23]:
%%time
ft_model.train(data, total_examples=ft_model.corpus_count, epochs=50, report_delay=1)

Wall time: 7min 42s


(56594883, 66831650)

### 4. Построение индексов

In [24]:
import annoy
from tqdm import tqdm_notebook

In [25]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

In [26]:
index_map = {}
counter = 0

for text in tqdm_notebook(data):
    n_w2v = 0
    n_ft = 0
    index_map[counter] = combine_df.to_numpy()[counter]
    
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in text:
        if word in w2v_model.wv:
            vector_w2v += w2v_model.wv[word]
            n_w2v += 1
        if word in ft_model.wv:
            vector_ft += ft_model.wv[word]
            n_ft += 1
    
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)
    
    counter += 1
        
    #if counter > 100000:
    #    break

w2v_index.build(10)
ft_index.build(10)

  0%|          | 0/198708 [00:00<?, ?it/s]

True

In [27]:
def get_response(input_text, count, index, model, index_map):
    print(f'input:\n{input_text}')
    input_text = preprocess_txt(input_text)
    vector = np.zeros(300)
    norm = 0
    for word in input_text:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    results = index.get_nns_by_vector(vector, count, )
    return [index_map[i] for i in results]

In [28]:
def get_response_w2v(input_text, count):
    return get_response(input_text, count, w2v_index, w2v_model, index_map)

In [29]:
def get_response_ft(input_text, count):
    return get_response(input_text, count, ft_index, ft_model, index_map)

In [30]:
results = get_response_w2v(combine_df[1], 5)
print(f'output:')
for output_text in results:
    print(output_text)

input:
Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D
output:
Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D
@Aleks_SPb @CIO_Optimal Просто боится потерять наворованное,нервничает и от этого ненавидит и гнобит всех и вся вокруг
@RobinFlamm64 я тоже офигела :DD у нее будет двойня
RT @nvtextile2012: Кстати, сегодня 11.12.13 ... взлетаем? :)
@KamilGaleev не думаю, что ты много потерял, не поставив номер на месте покупки)


In [31]:
results = get_response_ft(combine_df[1], 5)
print(f'output:')
for output_text in results:
    print(output_text)

input:
Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D
output:
Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D
Пора закалывать себя снова чтоб хоть как то сегодня вечером спортом заняться на морально волевых )))
@OlegOll Полностью согласна. Теоретически :) Только у меня сложности с практикой.. @e_sivanova
Эй, я слышала, что твоя пресс-конференция будет завтра. :~( #INSTANTFOLLOWBACK #FF_RU
@PolinaShlyaga да.преподов на кафедре бх вообще неи.только свергун бегает(


In [32]:
results = get_response_w2v(combine_df[10], 5)
print(f'output:')
for output_text in results:
    print(output_text)

input:
Люблю маму и папу!!!!а в остальное я так...-влюбляюсь, привязываюсь))) 
output:
Люблю маму и папу!!!!а в остальное я так...-влюбляюсь, привязываюсь))) 
Лучше сейчас разорвать все отношения, пока не так больно((
Сидишь тут ради него до 3-х ночи,а он так тебя расстраивает.
Грустно прям(
Всем снов
@mishk_n @mudakoff слыыш хаха
я не пью хаах
вот сейчас бухают /празднуют др .а я тут дома .сочком:D
засыпаешь в темноте....просыпашься темно...вот такая жизнь(


In [33]:
results = get_response_ft(combine_df[10], 5)
print(f'output:')
for output_text in results:
    print(output_text)

input:
Люблю маму и папу!!!!а в остальное я так...-влюбляюсь, привязываюсь))) 
output:
Люблю маму и папу!!!!а в остальное я так...-влюбляюсь, привязываюсь))) 
@delanoo @Larleyt я уже с ноября не хожу на учёбу. Совсем забросил. Чую, сессия весёлая будет :D
@DEW3L МЮ на выезде играет .Хорошо ответил ,я это знаю всё )Просто интересно было твоё мнение .Просто им эта победа та и ни к чему уже
@Witaless ахахаахахах;)
Я не учил, не пойдём завтра на литературу
@danielsmoon Я тебе разведусь слышишь,эй :D 
Даже не думай,ты моя единственная любовь ;)


Видим достаточно неплохой результат у обеих моделей. Субъективно у Word2Vec результат несколько лучше. 